### some custom functions for evaluations

In [58]:
def acc_f1_cv_score(model,x_train,n_splits=5,y_train=y_train,random_state=42):
  """
  specify the name of x_train in the 2nd arg.
  specify the n_splits for the kfold(def is 5)
  y_train is by default with this object name(unless specified otherwise)[not changed a lot:dont have various object names]
  - set a fixed random_state for reproducibality
  returns accuracy and f1 cv scores
  """  
  strat_kfold=StratifiedKFold(n_splits=n_splits,shuffle=True,random_state=random_state)#n_splits must be odd num
  acc_score=cross_val_score(model,x_train,y_train,cv=strat_kfold,scoring="accuracy")
  f1_score=cross_val_score(model,x_train,y_train,cv=strat_kfold,scoring="f1")  
    
    
  return acc_score.mean(),f1_score.mean()

def acc_f1_score_train(model,x_train,y_train=y_train):
    """
    specify the name of x_train in the 2nd arg.
    y_train is by default with this object name(unless specified otherwise)[not changed a lot:dont have various object names]
    returns accuracy and f1 scores of predicting x_train
    """
    model.fit(x_train,y_train)
    y_train_pred=model.predict(x_train)
    
    return accuracy_score(y_train,y_train_pred),f1_score(y_train,y_train_pred)

def acc_f1_score_test(model,x_train,x_test, y_train=y_train ,y_test=y_test):
    """
    specify the name of x_train and x_test in the 2nd and 3rd argument
    -y_train and test is by default with this object name(unless specified otherwise)[not changed a lot:dont have various object names]
    returns accuracy and f1 scores of predicting x_test
    """
    model.fit(x_train,y_train)
    y_test_pred=model.predict(x_test)
    
    return accuracy_score(y_test,y_test_pred),f1_score(y_test,y_test_pred)

### saving results

In [62]:
models=[]
#poly=[]
def_tuned=[]#model with default params or tuned(usie GS/RS/manual Htn on cv)
pca=[]#applying pca or not
accuracy_lst=[]
f1_lst=[]
notes=[]
#5(or 6 appends if u want to add a norte column) appends for each model(we count def model and tuned model as diff models)
def appending(model, default_tuned, pca_no, accuracy, f1, note=None):#add poly list later(a list to specify if poly was applied and w what degree)
    '''
    *this function appends certain info about the models which we will zipped(or use dict of lists) to be converted to a dataframe and used for visualiztion*
    a-initialize the lists outside the function so that they arent reset each time u call the function or b-u can put them inside but make them *global variables* or c-add them as args and return them
    args:
    1-model:append model name to a list
    2-default_tuned:append to a list whether the model is tuned or not
    3-pca_no:whether pca was applied or not and num of components
    4-accuracy:append accuracy score to a list
    5-f1:append f1 score to a list
    6-note:to add a note about a certain model ,u can append it to a list
    '''
    #5(or 6 if u want to add a norte column) appends for each model(we count def model and tuned model as diff models)
    models.append(model)
    def_tuned.append(default_tuned)
    pca.append(pca_no)
    accuracy_lst.append(accuracy)
    f1_lst.append(f1)
    notes.append(note)

def generate_dataframe(models=models,def_tuned=def_tuned,pca=pca,accuracy_lst=accuracy_lst,f1_lst=f1_lst,notes=notes):
    '''
    This function generates a DataFrame from the lists after all appending is done.
    Returns:
    -DataFrame containing the model comparison data.
    '''
    # Combine all lists into a *dictionary of lists*
    zz = {
        'Model': models,
        'Tuned/Default': def_tuned,
        'PCA': pca,
        'Accuracy': accuracy_lst,
        'F1 Score': f1_lst,
        'Notes': notes
    }

    # Convert the dictionary into a pandas DataFrame
    zzz = pd.DataFrame(zz)
    return zzz


### example usage with logistic regression model

In [123]:
help(splitting_train_test)

Help on function splitting_train_test in module __main__:

splitting_train_test(x, y, encode_x=False, cat_x_encoding='onehot', encoding_map=None, ordinal_orders=None, scaling=False, poly_features=False, poly_degree=2, output_x_numpy=True, y_encoding='label', y_order=None)
    *This function is for preprocessing  classification target dataset*
    Args:
    1-x ( pd.DataFrame): Feature dataframe.(we will return it after split  as numpy.ndarray or as df [so that we can retain column names and indices for train,test datasets])
    2-y (pd.Series or np.array): Target variable (binary or multi-claas classification)(should be 1d array to be suitable for ML models.(use label or ordinal encoding or mapping)
    3-encode_x (bool)*[default is False]*: Whether to encode categorical features in x.(for most ML models u should encode features into numbers except:cat_boost for eg.)
    4-cat_x_encoding (str)[default is 'onehot']: Encoding type for categorical features. Choose from 'onehot', 'label', 

In [125]:
x,y=split_features_target(df, 'ejection fraction', print_check=False, object_as_cat=True)
x_train,x_test,y_train,y_test=splitting_train_test(x, y, encode_x=True, scaling =True ,output_x_numpy=True)#encode and scale features(x)
lo=LogisticRegression()
print(f'cv mean of accuaracy and f1 scores:{acc_f1_cv_score(lo,x_train)}')

cv mean of accuaracy and f1 scores:(0.6313754869643392, 0.6473071511451157)


In [131]:
x_train,x_test,y_train,y_test=splitting_train_test(x, y, encode_x=True,scaling =True,output_x_numpy=True)#encode and scale features(x)
lo=LogisticRegression()
acc_lo,f1_lo=acc_f1_score_test(lo, x_train, x_test)
print("accuracy score (test):",acc_lo)
print("f1 score (test):",f1_lo)
appending('LogisticRegression','default','no',acc_lo,f1_lo)

accuracy score (test): 0.550561797752809
f1 score (test): 0.5789473684210527


In [133]:
#with pca
x_train,x_test,y_train,y_test=splitting_train_test(x, y, encode_x=True, scaling =True ,output_x_numpy=True)#encode and scale features(x)
x_train_pca_9d, x_test_pca_9d, pca_9d, cum_sum_9d=plot_pca_variance_elbow(x_train, x_test, n_components=9, plot=False, interactive=False, figsize=(8, 5))
lo=LogisticRegression()
acc_lo2,f1_lo2=acc_f1_score_test(lo, x_train_pca_9d, x_test_pca_9d)
print("accuracy score (test):",acc_lo2)
print("f1 score (test):",f1_lo2)
appending("LogisticRegression",'default','yes,9',acc_lo2,f1_lo2)

PCA applied with 9 components.
accuracy score (test): 0.5561797752808989
f1 score (test): 0.582010582010582


### with this workflow u can automize preprocessing to your preference as well easily save ur results

---
---

### now u can display ur results to comapre between different models with different parameters and evaluation metrics and save it as an excel sheet or keep as DataFrame

In [211]:
summary=generate_dataframe()
# display(summary)

In [213]:
print(summary['PCA'].unique())
summary['PCA'] = summary['PCA'].replace( ['no', 'yes,9'],['no pca', 'pca 9'])
print(summary['Notes'].unique())
summary['encoding']=summary['Notes']
#.apply can only take one main if and else so ternary method will be used
#value_if_condition_true if condition else value_if_condition_false
summary['encoding'] = summary['encoding'].apply(
    lambda x: 'one-hot' if x is None 
    else ('label encoding' if x == 'used label encoding' 
          else ('no encoding and no split to train,test' if x == 'unsplit data' 
                else (None if x=='tuning hyperparameters except n_neighbors didnt have an impact' else x)))
)

display(summary)

['no' 'yes,9' 'pca cant work with catboost']
[None 'used label encoding'
 'tuning hyperparameters except n_neighbors didnt have an impact']


,Model,Tuned/Default,PCA,Accuracy,F1 Score,Notes,encoding
0,LogisticRegression,default,no pca,0.550562,0.578947,None,one-hot
1,LogisticRegression,default,pca 9,0.556180,0.582011,None,one-hot
2,LogisticRegression,tuned,no pca,0.556180,0.655022,None,one-hot
3,LogisticRegression,tuned,pca 9,0.556180,0.594872,None,one-hot
4,Decision Tree Classifier,default,no pca,0.528090,0.553191,None,one-hot
5,Decision Tree Classifier,default,pca 9,0.511236,0.524590,None,one-hot
6,Support Vector Classifier(SVC),default,no pca,0.561798,0.593750,None,one-hot
7,Support Vector Classifier(SVC),default,pca 9,0.544944,0.580311,None,one-hot
8,Support Vector Classifier(SVC),tuned,no pca,0.561798,0.632075,None,one-hot
9,Support Vector Classifier(SVC),tuned,pca 9,0.561798,0.632075,None,one-hot


In [221]:
#save the results as .csv file or excel sheet
import os
save_dir=os.path.join(r"C:\Users\MY PC\Documents\python_notebooks\projects\project 1","saved results.csv")
summary.to_csv(save_dir,index=False)